### Snscrape Attempt

In [ ]:
# ✅ Step 1: Install snscrape
!pip install snscrape

# ✅ Step 2: Define Ivy League accounts
ivy_accounts = [
    "@HarvardFootball", "@Coach_Joel_Lamb", "@crim_recruiting", "@Coach_Aurich", "@MicFein", "@ScottLarkee", "@Coach_Johnson76",
    "@yalefootball", "@Coach_smcgowan", "@StevenVashel", "@AlexKurtzYale", "@CoachJJAnderson", "@CoachRenoYale", "@paupaupau5",
    "@BrownU_Football", "@BrownHCPerry", "@CoachW_Edwards", "@CoachFrizBrownU", "@CoachEMorrissey", "@CoachDJackson1",
    "@PrincetonFTBL", "@CoachBobSurace", "@CoachFlinn", "@SVerbit", "@CoachCuevas78", "@CoachMikeWeick", "@coachehenderson",
    "@DartmouthFTBL", "@CoachDaft", "@coachirishodea", "@coachkeithclark", "@coach_dobes", "@CoachBracken",
    "@BigRed_Football", "@DanSwanstrom", "@TerryUrsin", "@CoachEFranklin",
    "@CULionsFB", "@CoachStoNGo", "@CoachSeitu", "@CoachManion_", "@Coach_Kukesh", "@SSmith_II",
    "@PennFB", "@CoachPriore", "@CoachDupont", "@CoachMetzler", "@coach_ru"
]

import os

# ✅ Step 3: Create a folder to store results
os.makedirs("ivy_mentions", exist_ok=True)

# ✅ Step 4: Scrape tweets (past year) for each Ivy account
for handle in ivy_accounts:
    clean_handle = handle.replace("@", "")
    print(f"Scraping tweets for: {handle}")
    !snscrape --jsonl --since 2024-07-10 twitter-search "{handle}" > ivy_mentions/{clean_handle}.json


Scraping tweets for: @HarvardFootball
2025-07-13 22:52:41.776  ERROR  snscrape.base  Error retrieving https://twitter.com/search?f=live&lang=en&q=%40HarvardFootball&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%40HarvardFootball&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
2025-07-13 22:52:41.776  CRITICAL  snscrape.base  4 requests to https://twitter.com/search?f=live&lang=en&q=%40HarvardFootball&src=spelling_expansion_revert_click failed, giving up.
2025-07-13 22:52:41.776  CRITICAL  snscrape.base  Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%40HarvardFootball&src=spelling_expansion_revert_click (Caused by SSLErr

In [ ]:
import json

unique_users = set()

for filename in os.listdir("ivy_mentions"):
    if filename.endswith(".json"):
        with open(f"ivy_mentions/{filename}", "r", encoding="utf-8") as f:
            for line in f:
                tweet = json.loads(line)
                username = tweet.get("user", {}).get("username")
                if username:
                    unique_users.add(username)

print(f"🔹 Total unique users: {len(unique_users)}")


In [ ]:
# Save to text file
with open("unique_users.txt", "w", encoding="utf-8") as f:
    for user in sorted(unique_users):
        f.write(user + "\n")

# Download to local machine
from google.colab import files
files.download("unique_users.txt")


Scraping tweets for: @HarvardFootball
Scraping tweets for: @Coach_Joel_Lamb
Scraping tweets for: @crim_recruiting
Scraping tweets for: @Coach_Aurich
Scraping tweets for: @MicFein
Scraping tweets for: @ScottLarkee
Scraping tweets for: @Coach_Johnson76
Scraping tweets for: @yalefootball
Scraping tweets for: @Coach_smcgowan
Scraping tweets for: @StevenVashel
Scraping tweets for: @AlexKurtzYale
Scraping tweets for: @CoachJJAnderson
Scraping tweets for: @CoachRenoYale
Scraping tweets for: @paupaupau5
Scraping tweets for: @BrownU_Football
Scraping tweets for: @BrownHCPerry
Scraping tweets for: @CoachW_Edwards
Scraping tweets for: @CoachFrizBrownU
Scraping tweets for: @CoachEMorrissey
Scraping tweets for: @CoachDJackson1
Scraping tweets for: @PrincetonFTBL
Scraping tweets for: @CoachBobSurace
Scraping tweets for: @CoachFlinn
Scraping tweets for: @SVerbit
Scraping tweets for: @CoachCuevas78
Scraping tweets for: @CoachMikeWeick
Scraping tweets for: @coachehenderson
Scraping tweets for: @Dartmou

In [ ]:
import json

unique_users = set()

for filename in os.listdir("ivy_mentions"):
    if filename.endswith(".json"):
        with open(f"ivy_mentions/{filename}", "r", encoding="utf-8") as f:
            for line in f:
                tweet = json.loads(line)
                username = tweet.get("user", {}).get("username")
                if username:
                    unique_users.add(username)

print(f"🔹 Total unique accounts: {len(unique_users)}")


🔹 Total unique accounts: 0


In [ ]:
with open("unique_users.txt", "w", encoding="utf-8") as out:
    for user in sorted(unique_users):
        out.write(user + "\n")

# Download the file if needed
from google.colab import files
files.download("unique_users.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Twitter API Attempt

In [ ]:
import requests
import pandas as pd
import time
from typing import List, Dict, Optional
from tqdm import tqdm

# ==== CONFIG ====
BEARER_TOKEN = "YOUR_TWITTER_BEARER_TOKEN"
SEARCH_KEYWORDS = ["Ivy League", "Yale football", "Harvard coach"]
ACCOUNTS = ["IvyLeague", "YaleAthletics", "HarvardFootball"]
START_DATE = "2023-07-01T00:00:00Z"
END_DATE = "2024-07-01T00:00:00Z"
MAX_RESULTS_PER_QUERY = 100  # Twitter API max per page
MAX_PAGES = 10
LANGUAGE = "en"
OUTPUT_FILE = "ivy_league_tweets_api.csv"


# ==== HEADERS ====
def create_headers(bearer_token: str) -> Dict[str, str]:
    return {"Authorization": f"Bearer {bearer_token}"}


# ==== QUERY BUILDER ====
def build_query(keyword: Optional[str] = None, account: Optional[str] = None, lang: str = "en") -> str:
    parts = []
    if keyword:
        parts.append(f'"{keyword}"')
    if account:
        parts.append(f'from:{account}')
    parts.append(f'lang:{lang}')
    return " ".join(parts)


# ==== API CALL FUNCTION ====
def fetch_tweets(query: str, start_time: str, end_time: str,
                 headers: Dict[str, str], max_pages: int = 10,
                 max_results: int = 100) -> List[Dict]:
    url = "https://api.twitter.com/2/tweets/search/all"
    params = {
        "query": query,
        "start_time": start_time,
        "end_time": end_time,
        "max_results": max_results,
        "tweet.fields": "id,text,author_id,created_at,lang,public_metrics,entities",
        "expansions": "author_id",
        "user.fields": "username,name,public_metrics",
    }

    results = []
    next_token = None

    for _ in tqdm(range(max_pages), desc=f"Query: {query[:40]}..."):
        if next_token:
            params["next_token"] = next_token

        response = requests.get(url, headers=headers, params=params)

        if response.status_code != 200:
            print(f"[ERROR] Status code {response.status_code}: {response.text}")
            break

        data = response.json()

        includes = {u['id']: u for u in data.get("includes", {}).get("users", [])}
        for tweet in data.get("data", []):
            user = includes.get(tweet["author_id"], {})
            metrics = tweet.get("public_metrics", {})
            results.append({
                "tweet_id": tweet["id"],
                "created_at": tweet["created_at"],
                "text": tweet["text"],
                "username": user.get("username"),
                "displayname": user.get("name"),
                "followersCount": user.get("public_metrics", {}).get("followers_count"),
                "retweets": metrics.get("retweet_count"),
                "likes": metrics.get("like_count"),
                "replies": metrics.get("reply_count"),
                "quotes": metrics.get("quote_count"),
                "hashtags": tweet.get("entities", {}).get("hashtags", []),
            })

        next_token = data.get("meta", {}).get("next_token")
        if not next_token:
            break
        time.sleep(1.2)  # be nice to the API

    return results


# ==== PIPELINE WRAPPER ====
def batch_api_scrape(keywords: List[str], accounts: List[str],
                     start_date: str, end_date: str,
                     bearer_token: str) -> pd.DataFrame:
    headers = create_headers(bearer_token)
    all_results = []

    for kw in keywords:
        for acc in accounts:
            query = build_query(keyword=kw, account=acc)
            tweets = fetch_tweets(query, start_date, end_date, headers)
            all_results.extend(tweets)

    return pd.DataFrame(all_results)


# ==== MAIN ====
if __name__ == "__main__":
    print("🚀 Starting Twitter API Scraping...\n")
    df = batch_api_scrape(SEARCH_KEYWORDS, ACCOUNTS, START_DATE, END_DATE, BEARER_TOKEN)

    print(f"\n✅ Done. Total tweets collected: {len(df)}")
    df.to_csv(OUTPUT_FILE, index=False)


🚀 Starting Twitter API Scraping...



Query: "Ivy League" from:IvyLeague lang:en...:   0%|          | 0/10 [00:00<?, ?it/s]


[ERROR] Status code 401: {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}


Query: "Ivy League" from:YaleAthletics lang:en...:   0%|          | 0/10 [00:00<?, ?it/s]


[ERROR] Status code 401: {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}


...

[ERROR] Status code 401: {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}


Query: "Harvard coach" from:YaleAthletics lang:...:   0%|          | 0/10 [00:00<?, ?it/s]


[ERROR] Status code 401: {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}


Query: "Harvard coach" from:HarvardFootball lan...:   0%|          | 0/10 [00:00<?, ?it/s]

[ERROR] Status code 401: {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}

✅ Done. Total tweets collected: 0
